In [3]:
import sys
import os
from timeit import default_timer as timer
os.getcwd()

'/home/jupyter/Notebooks/crystal/NLP/nlp_testing'

In [ ]:
# Adapted from the tutorial at https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/

In [4]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:
# list(tokenizer.vocab.keys())[5000:5020]
list(tokenizer.vocab.keys())[17733]

'disgusted'

In [ ]:
# Define a new example sentence with multiple meanings of the word "disgusted"
text = "[CLS] She made a disgusted pout [SEP] Her disgusted expression was contagious [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

In [ ]:
# Recreate vocabulary words from their tokenized representations.
for t in tokenized_text:
    this_word = ''
    for token in t:
        this_word += token.strip('#')
#     print(this_word)

In [ ]:
# Mark each of the tokens as belonging to sentence "0" or "1".

segments_ids = [1] * len(tokenized_text[3])
# segments_ids = [0,0,0]
print (segments_ids)
print(indexed_tokens)
print(tokenized_text[3])

In [ ]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens[3]])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [ ]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [ ]:
print ("Number of layers:", len(encoded_layers))
layer_i = 0

print ("Number of batches:", len(encoded_layers[layer_i]))
batch_i = 0

print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
token_i = 1

print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))

In [ ]:
# For our token, select its feature values from layer 5.
token_i = 1
layer_i = 5
vec = encoded_layers[layer_i][batch_i][token_i]
# print(vec)

# Plot the values as a histogram to show their distribution.
plt.figure(figsize=(10,10))
plt.hist(vec, bins=200)
plt.show()

In [ ]:
# `encoded_layers` is a Python list.
print('     Type of encoded_layers: ', type(encoded_layers))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', encoded_layers[0].size())

In [ ]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(encoded_layers, dim=0)

token_embeddings.size()

In [ ]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

In [ ]:
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

In [23]:
# Concatenate the last 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat_last = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_last.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_last), len(token_vecs_cat_last[0])))
print(token_vecs_cat_last[0])

Shape is: 1 x 3072
tensor([-0.2373,  0.8259, -0.6190,  ..., -0.3836, -0.5039,  0.6153])


In [24]:
# Sum the last 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_last = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_last.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_last), len(token_vecs_sum_last[0])))

Shape is: 1 x 768


In [ ]:
# Concatenate the first 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat_first = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[0], token[1], token[2], token[3]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_first.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_first), len(token_vecs_cat_first[0])))
print(token_vecs_cat[0])

In [ ]:
# Sum the first 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_first = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[:4], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_first.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_first), len(token_vecs_sum_first[0])))

In [ ]:
# Concatenate the middle 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat_middle1 = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[4], token[5], token[6], token[7]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_middle1.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_middle1), len(token_vecs_cat_middle1[0])))
print(token_vecs_cat[0])

In [ ]:
# Sum the middle 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_middle1 = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[4:8], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_middle1.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_middle1), len(token_vecs_sum_middle1[0])))

In [ ]:
# Concatenate the middle 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat_middle2 = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[8], token[9], token[10], token[11]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_middle2.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_middle2), len(token_vecs_cat_middle2[0])))
print(token_vecs_cat[0])

In [ ]:
# Sum the middle 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_middle2 = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[8:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_middle2.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_middle2), len(token_vecs_sum_middle2[0])))

In [ ]:
# Concatenate all hidden layers to create word embeddings.
token_vecs_cat_all = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[0], token[1], token[2], token[3], token[4], token[5], token[6], token[7], token[8], token[9], token[10], token[11]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_all.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_all), len(token_vecs_cat_all[0])))
print(token_vecs_cat[0])

In [ ]:
# Sum all hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_all = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_all.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_all), len(token_vecs_sum_all[0])))

In [ ]:
# Make a single vector to represent the pair of sentences by averaging across tokens.
# `encoded_layers` has shape [12 x 1 x 22 x 768]
sentences_vec = []
# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = encoded_layers[11][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)
for s in sentence_embedding:
    sentences_vec.append(s)
print ("Our final sentence embedding vector of shape:", sentence_embedding.size())
print(sentence_embedding[767])
print(sentence_embedding[-1])
print(f'Shape of sentences vector is: {len(sentences_vec)}')
print(sentences_vec[-1])

In [ ]:
from scipy.spatial.distance import cosine
# Test the similarity of a word with itself.
# For words trained contextually, self-synonymy is less than 1.
similarity = 1 - cosine(token_vecs_cat[0], token_vecs_cat[0])
print(f'Similarity of {tokenized_text[8]} and {tokenized_text[8]} in token_vecs_cat is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum[4], token_vecs_sum[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum is: {similarity}')
# similarity = 1 - cosine(token_vecs_cat_first[4], token_vecs_cat_first[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_cat_first is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum_first[4], token_vecs_sum_first[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum_first is: {similarity}')
# similarity = 1 - cosine(token_vecs_cat_middle1[4], token_vecs_cat_middle1[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_cat_middle1 is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum_middle1[4], token_vecs_sum_middle1[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum_middle1 is: {similarity}')
# similarity = 1 - cosine(token_vecs_cat_middle2[4], token_vecs_cat_middle2[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_cat_middle2 is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum_middle2[4], token_vecs_sum_middle2[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum_middle2 is: {similarity}')
# similarity = 1 - cosine(token_vecs_cat_all[4], token_vecs_cat_all[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_cat_all is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum_all[4], token_vecs_sum_all[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum_all is: {similarity}')

In [6]:
######################################################################################
############## BEGIN TESTING STATIC CONTEXTUAL EMBEDDING CREATION ####################
######################################################################################

In [7]:
def make_vocab(vocab_file):
    # start = timer()
    vocab = []
    # vocab_file = '/home/jupyter/Notebooks/crystal/NLP/MiFace/Python/vocab_files/vocab_checked.txt'
    with open(vocab_file, 'r') as v:
        vocab = v.read().splitlines()
    # end = timer()
    # run_time = end - start
#     print(f'There are {len(vocab)} words in the vocabulary.\n')
#     print(f'It took {run_time} seconds to read the vocabulary file into memory.')
#     print(f'Test word is {vocab[2]}.')
    return vocab

In [8]:
def tokenize_text(vocab):
    tokenized_text = []
    indexed_tokens = []
    for word in vocab:
        # Add the special tokens.
    #     marked_text = "[CLS] " + word + " [SEP]"
        marked_text = word

        # Split the sentence into tokens.
        # tokenized_text = tokenizer.tokenize(marked_text)
        tokenized_text.append(tokenizer.tokenize(marked_text))
#         print(f'Added {tokenized_text[-1]} to the tokenized_text array.')


        # Map the token strings to their vocabulary indeces.
        indexed_tokens.append(tokenizer.convert_tokens_to_ids(tokenized_text[-1]))

        # Display the words with their indeces.
    #     print(f'The word {tokenized_text[-1][1]} is at index {indexed_tokens[-1]}.')
#         for tup in zip(tokenized_text[-1], indexed_tokens[-1]):
#             print('{:<12} {:>6,}'.format(tup[0], tup[1]))
    return tokenized_text, indexed_tokens

In [9]:
def generate_segments_IDs(tokenized_text):
    # Create segment IDs for sentence 1 (there can be a sentence 0 to compare to
    # sentence 1, but we're not doing that).
    # Check that indices and token indices look correct.
    segments_IDs = []
    for i in range(len(tokenized_text)):
        segments_IDs.append([1] * len(tokenized_text[i]))
#     for i in range(len(segments_IDs)):
#         print (segments_IDs[i])
#         print(tokenized_text[i])
    return segments_IDs

In [10]:
def generate_embeddings(indexed_tokens, segments_IDs):
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_IDs])

    # Load pre-trained model (weights)
    model = BertModel.from_pretrained('bert-base-uncased')

    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()

    # Predict hidden states features for each layer
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)
#         print('Type of encoded_layers: ', type(encoded_layers))
        # Each layer in the list is a torch tensor.
#         print('Tensor shape for each layer: ', encoded_layers[0].size())

    # Concatenate the tensors for all layers. We use `stack` here to
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(encoded_layers, dim=0)
#     print(token_embeddings.size())

    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
#     print(token_embeddings.size())

    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)
#     print(token_embeddings.size())
    
    return token_embeddings

In [11]:
def cat_last_four(token_embeddings): 
    # Concatenate the last 4 hidden layers to create contextual embeddings.
    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat_last = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat_last.append(cat_vec)

    print ('Shape is: %d x %d' % (len(token_vecs_cat_last), len(token_vecs_cat_last[0])))
    print(token_vecs_cat_last[0])
    
    return token_vecs_cat_last

In [12]:
def cat_middle_four2(token_embeddings):
    # Concatenate the middle 4 hidden layers to create word embeddings.
    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat_middle2 = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last 
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[8], token[9], token[10], token[11]), dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat_middle2.append(cat_vec)

    print ('Shape is: %d x %d' % (len(token_vecs_cat_middle2), len(token_vecs_cat_middle2[0])))
    print(token_vecs_cat_middle2[0])
    
    return token_vecs_cat_middle2

In [13]:
def cat_middle_four1(token_embeddings):
    # Concatenate the middle 4 hidden layers to create word embeddings.
    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat_middle1 = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last 
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[4], token[5], token[6], token[7]), dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat_middle1.append(cat_vec)

    print ('Shape is: %d x %d' % (len(token_vecs_cat_middle1), len(token_vecs_cat_middle1[0])))
    print(token_vecs_cat_middle1[0])
    
    return token_vecs_cat_middle1

In [14]:
def cat_first_four(token_embeddings):
    token_vecs_cat_first = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last 
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[0], token[1], token[2], token[3]), dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat_first.append(cat_vec)

    print ('Shape is: %d x %d' % (len(token_vecs_cat_first), len(token_vecs_cat_first[0])))
    print(token_vecs_cat_first[0])
    
    return token_vecs_cat_first

In [15]:
def get_mean_embedding(token_embeddings):
    mean_embedding = sum(token_embeddings) / len(token_embeddings)
    print(mean_embedding)
    
    return mean_embedding

In [16]:
def reconstruct_tokens(tokenized_text):
    vocab_word = ''
    for i in tokenized_text:
        vocab_word += i.strip('#')
    print(vocab_word)

    return vocab_word

In [17]:
def write_embedding(embeddings_file, vocab_word, contextual_embedding):
    try:
        with open(embeddings_file, 'a') as f:
            f.write(vocab_word)
            for value in contextual_embedding[0]:
                f.write(' ' + str(value.item()))
            f.write('\n')
        print(f'Saved the embedding for {vocab_word}.')
    except:
        print('Oh no! Unable to write to the embeddings file.')

In [ ]:
# if __name__ == "__main__":
# Set up input and output paths.
vocab_file = '/home/jupyter/Notebooks/crystal/NLP/MiFace/Python/vocab_files/vocab_checked.txt'
layer_combining_function = cat_middle_four2
embeddings_file = os.path.join('/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab', layer_combining_function.__name__ + '.txt')
if os.path.exists(embeddings_file):
    os.remove(embeddings_file)

    # Create a list of vocabulary words we want embeddings for.
vocab = make_vocab(vocab_file)

# Tokenize the vocabulary and look up the BERT token indices.
tokenized_text, indexed_tokens = tokenize_text(vocab)

# Generate segment IDs for each token.
segments_IDs = generate_segments_IDs(tokenized_text)

# Generate and write out the contextual embeddings for the vocabulary words.
# Embeddings are saved in a standard format that can be used for calcualting
# the cosine distances between word vectors.
for i in range(len(tokenized_text)):
    # Convert indexed tokens and segments to tensors.
    # Create a BERT model for the tokens.
    # Get the encoded model layers and reshape them.
    token_embeddings = generate_embeddings(indexed_tokens[i], segments_IDs[i])
    print(f'{tokenized_text[i]} has a token embedding of size {token_embeddings.size()}')

    # Extract the contextual embedding for a token.
    contextual_embedding = layer_combining_function(token_embeddings)

    # Write the embedding to a text file, with the vocabulary word prepended.
    vocab_word = reconstruct_tokens(tokenized_text[i])
    # Make sure we've got the correct vocabulary word.
    assert vocab[i] == vocab_word
    write_embedding(embeddings_file, vocab[i], contextual_embedding)

['aback'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.0913,  1.4276,  0.4738,  ..., -0.4144,  0.0285, -0.0063])
aback
Saved the embedding for aback.
['aba', '##shed'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.0985, -0.3056, -0.0392,  ...,  0.3008, -0.5077,  0.2036])
abashed
Saved the embedding for abashed.
['ab', '##hor'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.4193,  0.4612,  0.7739,  ..., -0.1391,  0.0203,  0.4971])
abhor
Saved the embedding for abhor.
['ab', '##hor', '##red'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([-0.0553,  0.5331,  0.3810,  ..., -0.1551, -0.0535,  0.1270])
abhorred
Saved the embedding for abhorred.
['ab', '##hor', '##rence'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.1730, -0.8257,  0.6310,  ..., -0.2092,  0.0052,  0.4344])
abhorrence
Saved the embedding 

['ag', '##gra', '##vation'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([-0.0653,  0.8995,  0.5749,  ...,  0.1177,  0.1458, -0.2573])
aggravation
Saved the embedding for aggravation.
['aggression'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.5144,  1.0312, -0.6284,  ...,  0.3372, -0.1049, -0.3468])
aggression
Saved the embedding for aggression.
['aggressive'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.1463,  0.5762, -0.7487,  ...,  0.3911, -0.5970, -0.5397])
aggressive
Saved the embedding for aggressive.
['ag', '##gr', '##ie', '##ve'] has a token embedding of size torch.Size([4, 12, 768])
Shape is: 4 x 3072
tensor([-0.0470,  0.4448,  0.8178,  ...,  0.1632,  0.1066, -0.7245])
aggrieve
Saved the embedding for aggrieve.
['ag', '##gr', '##ie', '##ved'] has a token embedding of size torch.Size([4, 12, 768])
Shape is: 4 x 3072
tensor([-0.4535,  0.2086,  1.0646,  ..., -0.295

['ang', '##st'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.1915,  0.0732,  0.7304,  ...,  0.6035,  0.2142, -0.1650])
angst
Saved the embedding for angst.
['anguish'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.0282,  1.4766,  0.3311,  ..., -0.4970,  0.7294, -0.4133])
anguish
Saved the embedding for anguish.
['anguish', '##ed'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.5110,  1.3244,  0.2221,  ..., -0.1738,  0.5002, -0.0183])
anguished
Saved the embedding for anguished.
['animated'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.4838,  0.6223,  0.4137,  ...,  0.0671,  0.9273, -0.0550])
animated
Saved the embedding for animated.
['an', '##imo', '##sity'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.7939,  0.5979,  0.1296,  ..., -0.5694, -0.1118,  0.1823])
animosity
Saved the embedding fo

['ashamed'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.8249,  0.6267, -0.9373,  ...,  1.0249,  0.1462, -0.2750])
ashamed
Saved the embedding for ashamed.
['aspiring'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-1.0652,  0.2626, -0.0492,  ...,  0.3584, -0.1985,  0.6254])
aspiring
Saved the embedding for aspiring.
['assert', '##ive'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.5699,  0.8623,  0.2104,  ...,  0.1907,  0.5752, -0.1986])
assertive
Saved the embedding for assertive.
['assert', '##ively'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.8104,  0.8994,  0.6644,  ..., -0.6660,  0.9689, -0.9693])
assertively
Saved the embedding for assertively.
['assessing'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.1620,  0.6290, -0.2597,  ...,  0.4135,  0.6680, -0.0492])
assessing
Saved the embedd

['beat'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.1441,  1.2114, -0.7565,  ...,  0.5628,  1.0945,  0.3002])
beat
Saved the embedding for beat.
['beaten'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-1.2069,  0.8025, -0.1612,  ..., -0.1613,  0.3740, -0.8912])
beaten
Saved the embedding for beaten.
['bed', '##ev', '##iled'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([-0.0442,  0.1295,  0.6587,  ..., -0.0631,  0.0515,  0.0749])
bedeviled
Saved the embedding for bedeviled.
['be', '##fu', '##ddled'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.0779,  0.1986,  0.8141,  ...,  0.0695,  0.0980, -0.0476])
befuddled
Saved the embedding for befuddled.
['begging'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.1176,  1.5832, -1.7210,  ...,  0.7771,  1.2993,  0.1483])
begging
Saved the embedding for beggi

...

['res', '##ent'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.8119,  0.7059,  0.4224,  ...,  0.5156,  0.3991,  0.3232])
resent
Saved the embedding for resent.
['res', '##ent', '##ful'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([-0.4961,  0.6787,  1.1469,  ..., -0.0020, -0.2213,  0.2547])
resentful
Saved the embedding for resentful.
['res', '##enting'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.0850,  0.7135,  1.1565,  ...,  0.0873, -0.2477, -0.4359])
resenting
Saved the embedding for resenting.
['resentment'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.1651,  0.3080, -0.6607,  ..., -0.3694,  0.1074, -0.3617])
resentment
Saved the embedding for resentment.
['reserved'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.9024,  0.1464, -0.2234,  ...,  0.4444,  0.6599, -0.1958])
reserved
Saved the

['rude'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.2743,  1.4921, -1.3118,  ...,  0.6078,  0.0542,  0.3155])
rude
Saved the embedding for rude.
['rue', '##ful'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.1947,  0.0231, -0.4452,  ..., -0.1316,  0.2934, -0.5996])
rueful
Saved the embedding for rueful.
['ru', '##ffled'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.2122, -0.2337,  0.2148,  ...,  0.1517, -0.0937,  0.1688])
ruffled
Saved the embedding for ruffled.
['rum', '##inating'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.6920,  0.6397, -0.2072,  ...,  0.7198,  0.3160,  0.3517])
ruminating
Saved the embedding for ruminating.
['rust', '##led'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.1814,  1.0177,  0.4662,  ..., -0.1460,  0.3409, -0.2930])
rustled
Saved the embedding for rustled.

['sc', '##orn', '##ful'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([-0.6392,  0.6409,  0.6498,  ...,  0.3199, -0.0337,  0.1653])
scornful
Saved the embedding for scornful.
['scowl'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.4755,  1.5118, -0.2807,  ..., -0.9004,  0.5228, -0.3453])
scowl
Saved the embedding for scowl.
['scowl', '##ing'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.2111,  1.2100,  0.0470,  ..., -0.1680,  0.5137,  0.0280])
scowling
Saved the embedding for scowling.
['scream'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([0.2164, 0.6354, 0.0186,  ..., 0.0978, 0.3162, 0.3296])
scream
Saved the embedding for scream.
['screaming'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.0957,  0.5750, -0.0155,  ...,  0.2512,  0.4541,  0.3795])
screaming
Saved the embedding for screaming.
['sc'

['shocking', '##ly'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.1134,  1.5751, -0.3316,  ...,  0.5579,  0.5694, -0.3242])
shockingly
Saved the embedding for shockingly.
['shook'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.2943,  0.8700, -1.8979,  ...,  0.4321,  1.0311, -0.5007])
shook
Saved the embedding for shook.
['shout'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.2155,  0.7258,  0.0052,  ...,  0.4876,  1.2998, -0.5576])
shout
Saved the embedding for shout.
['shouting'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.1856,  1.2181, -0.4316,  ...,  0.0904,  0.3564,  0.6262])
shouting
Saved the embedding for shouting.
['sh', '##rew', '##d'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([-0.0278,  0.4411,  0.4081,  ..., -0.1615,  0.1131, -0.4956])
shrewd
Saved the embedding for shrewd.
['shy

['smashed'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.2279,  0.5602, -0.1967,  ...,  0.1943,  1.4270, -0.0475])
smashed
Saved the embedding for smashed.
['smile'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-1.2074,  0.2163,  0.1038,  ...,  0.8780,  0.2743,  0.2939])
smile
Saved the embedding for smile.
['smiley'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.5748, -0.6122, -0.2743,  ...,  0.4419, -0.3123,  0.1988])
smiley
Saved the embedding for smiley.
['smiling'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.4768,  0.6892, -1.1503,  ...,  0.4351,  0.4747, -0.0259])
smiling
Saved the embedding for smiling.
['smirk'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.4651,  1.0724, -0.6079,  ..., -0.8278,  0.1095, -0.8049])
smirk
Saved the embedding for smirk.
['smirk', '##ing'] has a token embe